# Data Preparation and Merging Different Datasets

## Libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder                                           
from sklearn.model_selection import train_test_split                                               
from sklearn.preprocessing import StandardScaler                                                      
import warnings                                                                                        
warnings.filterwarnings('ignore')
import os


## Master Dataset 1 Merging Carbon Policy Dataset files (4 Core files)
  - General Info (Compliance_Gen_Info.csv)
  - Emissions Coverage (Compliance_Emissions.csv)
  - Carbon Price (Compliance_Price.csv)
  - Revenue (Compliance_Revenue.csv)


### Loading Dataset

In [2]:
gen_info = pd.read_csv('../dataset/dataset converted to csv format/data_08_2025_Compliance_Gen_Info.csv', skiprows=4)
emissions = pd.read_csv('../dataset/dataset converted to csv format/data_08_2025_Compliance_Emissions.csv', skiprows=2)
price = pd.read_csv('../dataset/dataset converted to csv format/data_08_2025_Compliance_Price.csv', skiprows=1)
revenue = pd.read_csv('../dataset/dataset converted to csv format/data_08_2025_Compliance_Revenue.csv', skiprows=1)

Shapes of the datasets

In [3]:
print(f"General Info: {gen_info.shape}")


General Info: (131, 44)


In [4]:
print(f"Emissions: {emissions.shape}")

Emissions: (91, 37)


In [5]:
print(f"Price: {price.shape}")


Price: (80, 42)


In [6]:
print(f"Revenue: {revenue.shape}")


Revenue: (92, 41)


Column Names

In [7]:
print("Gen Info:", list(gen_info.columns[:5]))


Gen Info: ['Unique ID', 'Instrument name', 'Type', 'Status', 'Jurisdiction covered']


In [8]:
print("Emissions:", list(emissions.columns[:5]))


Emissions: ['Name of the initiative', '1990.0', '1991.0', '1992.0', '1993.0']


In [9]:
print("Price:", list(price.columns[:5]))


Price: ['Unique ID', 'Name of the initiative', 'Instrument Type', 'Region', 'Income group']


In [10]:
print("Revenue:", list(revenue.columns[:5]))


Revenue: ['Instrument name', 'Jurisdiction', 'Status', 'Type', 'Country income group']


### EXPLORATORY DATA ANALYSIS


In [11]:
datasets = {'Gen Info': gen_info, 'Emissions': emissions, 'Price': price, 'Revenue': revenue}


#### Missing Values Analysis

In [12]:
for name, df in datasets.items():
    total_cells = df.shape[0] * df.shape[1]
    missing_cells = df.isnull().sum().sum()
    print(f"{name}: {missing_cells:,} missing ({missing_cells/total_cells*100:.1f}%)")
    
    missing_by_col = df.isnull().sum()
    top_missing = missing_by_col[missing_by_col > 0].head(3)
    if not top_missing.empty:
        print(f"   Top missing columns: {dict(top_missing)}")

Gen Info: 2,568 missing (44.6%)
   Top missing columns: {'Share of jurisdiction emissions covered': 51, 'Price on 1 April': 51, '2020': 80}
Emissions: 0 missing (0.0%)
Price: 2,060 missing (61.3%)
   Top missing columns: {'1990.0': 80, '1991.0': 80, '1992.0': 80}
Revenue: 2,242 missing (59.4%)
   Top missing columns: {'1990.0': 90, '1991.0': 88, '1992.0': 88}


#### Missing Values Interpretation

##### Gen Info (44.6% missing):
This is normal for metadata fields which are often incomplete. The missing values in 'Share of jurisdiction emissions covered' and 'Price on 1 April' reflect that not all policies report these details, and only active policies have current prices. Missing data reflects real-world policy reporting gaps.

##### Emissions (0% missing):
Excellent - we have complete time series data. Every policy has emission coverage data for all years. The zeros in this dataset are meaningful, indicating that a policy exists but covers 0% of emissions.

##### Price (61.3% missing):
This is expected for carbon pricing history. The 1990s data is missing because very few carbon pricing policies existed then. Missing data does not indicate poor quality but reflects historical reality as most carbon pricing started post-2000.

##### Revenue (59.4% missing):
This is realistic for government revenue data. Early years (1990s) show missing data because there was no carbon pricing and therefore no revenue. Missing data shows when policies actually started generating revenue. Recent years have much better coverage.

##### Key Insight:
Missing values follow historical patterns, not data quality issues. The 1990s had few or no carbon pricing policies globally, the 2000s were an early adoption period, and 2010s onward saw rapid expansion of carbon pricing.




Policy names comparison for matching across datasets


In [13]:
gen_info_names = set(gen_info['Instrument name'].dropna())
emissions_names = set(emissions['Name of the initiative'].dropna())
price_names = set(price['Name of the initiative'].dropna())
revenue_names = set(revenue['Instrument name'].dropna())

In [14]:
print(f"Unique policy names:")
print(f"Gen Info: {len(gen_info_names)} policies")
print(f"Emissions: {len(emissions_names)} policies")
print(f"Price: {len(price_names)} policies")
print(f"Revenue: {len(revenue_names)} policies")

Unique policy names:
Gen Info: 131 policies
Emissions: 91 policies
Price: 80 policies
Revenue: 92 policies


In [15]:
gen_emissions_overlap = gen_info_names.intersection(emissions_names)
gen_price_overlap = gen_info_names.intersection(price_names)
gen_revenue_overlap = gen_info_names.intersection(revenue_names)

In [16]:
print(f"Policy name overlaps:")
print(f"Gen Info & Emissions: {len(gen_emissions_overlap)} matches")
print(f"Gen Info & Price: {len(gen_price_overlap)} matches")
print(f"Gen Info & Revenue: {len(gen_revenue_overlap)} matches")

Policy name overlaps:
Gen Info & Emissions: 90 matches
Gen Info & Price: 78 matches
Gen Info & Revenue: 90 matches


In [17]:
print("Example matching names (first 5):")
print(list(gen_emissions_overlap)[:5])

Example matching names (first 5):
['China national ETS', 'Oregon ETS', 'Estonia carbon tax', 'Washington CCA', 'Tamaulipas carbon tax']


In [18]:
print("Policies only in Gen Info (first 5):")
only_gen = gen_info_names - emissions_names - price_names - revenue_names
print(list(only_gen)[:5])

Policies only in Gen Info (first 5):
['India ETS', 'Thailand carbon tax', 'Colombia ETS', 'Pennsylvania ETS', 'Thailand ETS']


In [19]:
print("Policies only in Emissions (first 5):")
only_emissions = emissions_names - gen_info_names
print(list(only_emissions)[:5])

Policies only in Emissions (first 5):
['Taiwan carbon fee']


In [20]:
print("Policies only in Price (first 5):")
only_price = price_names - gen_info_names
print(list(only_price)[:5])

Policies only in Price (first 5):
['Manitoba carbon tax', 'Shandong ETS']


In [21]:
all_four = gen_info_names.intersection(emissions_names).intersection(price_names).intersection(revenue_names)
print(f"Policies in ALL 4 datasets: {len(all_four)}")
print("Examples:", list(all_four)[:5])

Policies in ALL 4 datasets: 78
Examples: ['China national ETS', 'Oregon ETS', 'Estonia carbon tax', 'Washington CCA', 'Tamaulipas carbon tax']


In [22]:
at_least_three = []
for policy in gen_info_names:
    count = 0
    if policy in emissions_names: count += 1
    if policy in price_names: count += 1
    if policy in revenue_names: count += 1
    if count >= 2:  # Gen Info + at least 2 others
        at_least_three.append(policy)

print(f"\nPolicies in Gen Info + at least 2 others: {len(at_least_three)}")


Policies in Gen Info + at least 2 others: 90


#### Policy Selection and Merging Strategy

##### Decision: Conservative Approach
We selected to keep only the 78 policies that appear in all 4 datasets. This ensures complete data coverage for both environmental and economic modeling without complex missing value handling.

##### Rationale:
- Complete data for all simulation types (emissions, prices, revenue)
- No gaps to fill or estimate
- Sufficient dataset size for machine learning (78 policies)
- High data quality and reliability
- Simpler preprocessing pipeline

##### Safe Merging Approach:
Before merging, we verify that policy names match exactly across all datasets and check for duplicates. We merge step-by-step with integrity checks at each stage to ensure no data loss or corruption. This creates a master dataset combining policy metadata, emission coverage time series, price time series, and revenue time series.

##### Expected Outcome:
One unified dataset with 78 policies and complete information across all dimensions needed for EcoImpact AI simulation.


In [23]:
core_policies = list(all_four)
print(f"Core policies selected: {len(core_policies)}")

Core policies selected: 78


In [24]:
gen_info_filtered = gen_info[gen_info['Instrument name'].isin(core_policies)]
emissions_filtered = emissions[emissions['Name of the initiative'].isin(core_policies)]
price_filtered = price[price['Name of the initiative'].isin(core_policies)]
revenue_filtered = revenue[revenue['Instrument name'].isin(core_policies)]



In [25]:
print(f"Filtered dataset shapes:")
print(f"Gen Info: {gen_info_filtered.shape}")
print(f"Emissions: {emissions_filtered.shape}")
print(f"Price: {price_filtered.shape}")
print(f"Revenue: {revenue_filtered.shape}")

Filtered dataset shapes:
Gen Info: (78, 44)
Emissions: (78, 37)
Price: (78, 42)
Revenue: (78, 41)


### Merging Datasets


In [26]:
gen_names = set(gen_info_filtered['Instrument name'])
emissions_names = set(emissions_filtered['Name of the initiative'])
price_names = set(price_filtered['Name of the initiative'])
revenue_names = set(revenue_filtered['Instrument name'])

In [27]:
print("Policy name verification:")
print(f"Gen Info policies: {len(gen_names)}")
print(f"Emissions policies: {len(emissions_names)}")
print(f"Price policies: {len(price_names)}")
print(f"Revenue policies: {len(revenue_names)}")


Policy name verification:
Gen Info policies: 78
Emissions policies: 78
Price policies: 78
Revenue policies: 78


In [28]:
assert gen_names == emissions_names, "Gen Info and Emissions names don't match!"
assert gen_names == price_names, "Gen Info and Price names don't match!"
assert gen_names == revenue_names, "Gen Info and Revenue names don't match!"
print("All policy names match exactly across datasets")


All policy names match exactly across datasets


In [29]:
for name, df, col in [('Gen Info', gen_info_filtered, 'Instrument name'),
                      ('Emissions', emissions_filtered, 'Name of the initiative'),
                      ('Price', price_filtered, 'Name of the initiative'),
                      ('Revenue', revenue_filtered, 'Instrument name')]:
    duplicates = df[col].duplicated().sum()
    print(f"{name} duplicates: {duplicates}")
    assert duplicates == 0, f"Found duplicates in {name}!"

print(" No duplicate policies within datasets")

Gen Info duplicates: 0
Emissions duplicates: 0
Price duplicates: 0
Revenue duplicates: 0
 No duplicate policies within datasets


In [30]:
original_counts = {
    'gen_info': len(gen_info_filtered),
    'emissions': len(emissions_filtered),
    'price': len(price_filtered),
    'revenue': len(revenue_filtered)
}
print(f"Original row counts: {original_counts}")

Original row counts: {'gen_info': 78, 'emissions': 78, 'price': 78, 'revenue': 78}


Merging

In [31]:
master_df = gen_info_filtered.copy()
print(f"Step 1 - Base (Gen Info): {master_df.shape}")

Step 1 - Base (Gen Info): (78, 44)


In [32]:
master_df = master_df.merge(
    emissions_filtered, 
    left_on='Instrument name', 
    right_on='Name of the initiative', 
    how='inner'
)
print(f"Step 2 - After Emissions merge: {master_df.shape}")
assert len(master_df) == original_counts['gen_info'], "Lost rows in Emissions merge!"

Step 2 - After Emissions merge: (78, 81)


In [33]:
master_df = master_df.merge(
    price_filtered, 
    left_on='Instrument name', 
    right_on='Name of the initiative', 
    how='inner',
    suffixes=('', '_price')
)
print(f"Step 3 - After Price merge: {master_df.shape}")
assert len(master_df) == original_counts['gen_info'], "Lost rows in Price merge!"


Step 3 - After Price merge: (78, 123)


In [34]:
master_df = master_df.merge(
    revenue_filtered, 
    left_on='Instrument name', 
    right_on='Instrument name',  
    how='inner',
    suffixes=('', '_revenue')
)
print(f"Step 4 - After Revenue merge: {master_df.shape}")
assert len(master_df) == original_counts['gen_info'], "Lost rows in Revenue merge!"

print("All merges completed successfully with no data loss")

Step 4 - After Revenue merge: (78, 163)
All merges completed successfully with no data loss


In [35]:
print(f"\nFinal master dataset shape: {master_df.shape}")
print(f"Total columns: {len(master_df.columns)}")


Final master dataset shape: (78, 163)
Total columns: 163


In [36]:
print("Sample merged data:")
print(master_df[['Instrument name', 'Type', 'Status']].head(3))

Sample merged data:
      Instrument name        Type       Status
0  Albania carbon tax  Carbon tax  Implemented
1  Alberta carbon tax  Carbon tax    Abolished
2        Alberta TIER         ETS  Implemented


In [37]:
os.makedirs('../dataset/processed', exist_ok=True)


In [38]:
output_path = '../dataset/processed/master_dataset.csv'
master_df.to_csv(output_path, index=False)
print(f"Master dataset saved to: {output_path}")


Master dataset saved to: ../dataset/processed/master_dataset.csv


In [39]:
from datetime import datetime
timestamp = datetime.now().strftime("%Y%m%d_%H%M")
backup_path = f'../dataset/processed/master_dataset_backup_{timestamp}.csv'
master_df.to_csv(backup_path, index=False)
print(f"Backup saved to: {backup_path}")

Backup saved to: ../dataset/processed/master_dataset_backup_20251215_2248.csv


In [40]:
summary_info = {
    'creation_date': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    'total_policies': len(master_df),
    'total_columns': len(master_df.columns),
    'source_datasets': ['Gen_Info', 'Emissions', 'Price', 'Revenue'],
    'policy_selection': 'Policies present in all 4 datasets',
    'missing_values': master_df.isnull().sum().sum()
}
print(f"\nDataset Summary:")
for key, value in summary_info.items():
    print(f"  {key}: {value}")


Dataset Summary:
  creation_date: 2025-12-15 22:48:36
  total_policies: 78
  total_columns: 163
  source_datasets: ['Gen_Info', 'Emissions', 'Price', 'Revenue']
  policy_selection: Policies present in all 4 datasets
  missing_values: 4601


Master dataset created by merging 4 core datasets. Contains 78 policies with complete environmental and economic data.


### Master Dataset

#### Loading Master Dataset

In [41]:
master_df = pd.read_csv('../dataset/processed/master_dataset.csv')


In [42]:
print(f"Shape: {master_df.shape}")
print(f"Policies: {len(master_df)}")
print(f"Features: {len(master_df.columns)}")

Shape: (78, 163)
Policies: 78
Features: 163


In [43]:
master_df_columns = master_df.columns.tolist()
print(f"Columns: {master_df_columns}")

Columns: ['Unique ID', 'Instrument name', 'Type', 'Status', 'Jurisdiction covered', 'Share of jurisdiction emissions covered', 'Price on 1 April', '2020', '2021', '2022', '2023', '2024', '2025', 'Change', 'Government revenue', '2019', '2020.1', '2021.1', '2022.1', '2023.1', '2024.1', 'Change.1', 'Gases covered', 'Electricity and heat', 'Industry', 'Mining and extractives', 'Transport', 'Aviation', 'Buildings', 'Agriculture, forestry and fishing fuel use', 'Agricultural emissions', 'Waste', 'LULUCF', 'Fuels covered', 'Allocation approaches', 'Price or market management', 'Point of Regulation', 'Offset eligibility', 'Description', 'Recent developments', 'Coverage', 'Pricing and allocation', 'Compliance', 'Relation to other instruments', 'Name of the initiative', '1990.0', '1991.0', '1992.0', '1993.0', '1994.0', '1995.0', '1996.0', '1997.0', '1998.0', '1999.0', '2000.0', '2001.0', '2002.0', '2003.0', '2004.0', '2005.0', '2006.0', '2007.0', '2008.0', '2009.0', '2010.0', '2011.0', '2012.0',

#### EDA on Master Dataser

In [44]:
missing_summary = master_df.isnull().sum()
missing_pct = (missing_summary / len(master_df)) * 100
missing_df = pd.DataFrame({
    'Missing Count': missing_summary,
    'Missing %': missing_pct
}).sort_values('Missing Count', ascending=False)


In [45]:
print(f"Top 10 columns with missing values:")
print(missing_df[missing_df['Missing Count'] > 0].head(10))

print(f"\nData types distribution:")
print(master_df.dtypes.value_counts())

Top 10 columns with missing values:
                Missing Count   Missing %
1991.0_price               78  100.000000
1990.0_price               78  100.000000
1992.0_price               78  100.000000
1993.0_price               78  100.000000
1997.0_price               76   97.435897
1990.0_revenue             76   97.435897
1993.0_revenue             75   96.153846
1992.0_revenue             75   96.153846
1994.0_price               75   96.153846
1991.0_revenue             75   96.153846

Data types distribution:
float64    119
object      44
Name: count, dtype: int64


What the Missing Values Mean:<br>
1990s Price Data (100% missing): No carbon pricing existed globally in early 1990s<br>
1990s Revenue Data (97% missing): No carbon pricing = no government revenue<br>
This reflects historical reality, not data quality issues<br>
Key Insights:<br>
-Expected Pattern: Carbon pricing started post-2000, so early years are naturally missing  <br>
-Data Quality: 119 numerical columns + 44 categorical = comprehensive dataset<br>
 -No Action Needed: Missing values are historically accurate

In [46]:
categorical_cols = ['Type', 'Status', 'Region', 'Income group', 'Jurisdiction covered']

print("")
for col in categorical_cols:
    if col in master_df.columns:
        print(f"\n")
        value_counts = master_df[col].value_counts()
        print(value_counts)
        print(f"Unique values: {master_df[col].nunique()}")
        print(f"Missing: {master_df[col].isnull().sum()}")




Type
Carbon tax    39
ETS           39
Name: count, dtype: int64
Unique values: 2
Missing: 0


Status
Implemented    67
Abolished      11
Name: count, dtype: int64
Unique values: 2
Missing: 0


Region
Europe & Central Asia         27
North America                 22
East Asia & Pacific           18
Latin America & Caribbean      9
Middle East & North Africa     1
Sub-Saharan Africa             1
Name: count, dtype: int64
Unique values: 6
Missing: 0


Income group
High income            56
Upper middle income    22
Name: count, dtype: int64
Unique values: 2
Missing: 0


Jurisdiction covered
Mexico                       2
British Columbia             2
Alberta                      2
Newfoundland and Labrador    2
Ontario                      2
                            ..
Korea, Rep.                  1
Latvia                       1
Liechtenstein                1
Luxembourg                   1
Zacatecas                    1
Name: count, Length: 68, dtype: int64
Unique values: 68
Mis

##### Categorical Features Interpretation

##### Type (Perfect Balance):
- **Carbon Tax**: 39 policies
- **ETS**: 39 policies
- **Great for ML**: Balanced dataset, no bias toward one policy type

##### Status (Mostly Successful):
- **Implemented**: 67 policies (86%) - Most policies succeed
- **Abolished**: 11 policies (14%) - Some failures to learn from

##### Region (Good Global Coverage):
- **Europe & Central Asia**: 27 policies (largest)
- **North America**: 22 policies
- **East Asia & Pacific**: 18 policies
- **Others**: Smaller representation

##### Jurisdiction (Too Granular):
- **68 unique jurisdictions** - Too many categories for ML
- **Need to group** by region or income level for better model performance

##### Key Insights:
- Balanced policy types enable unbiased learning
- High implementation success rate (86%) shows policy viability
- Good regional diversity for global applicability
- Will use Region instead of specific Jurisdiction for modeling


Time Series Analysis - Full 1990-2025 Timeline

In [ ]:
import re

def extract_year_columns(df):
    year_columns = []
    for col in df.columns:
        if re.match(r'^(19|20)\d{2}$', str(col)):  
            year_columns.append((col, int(col)))
        elif re.match(r'^(19|20)\d{2}\.0$', str(col)):  
            year = int(float(col))
            year_columns.append((col, year))
        elif re.match(r'^(19|20)\d{2}\.0_\w+$', str(col)):  
            year_part = col.split('.0_')[0]
            year = int(year_part)
            year_columns.append((col, year))
    year_columns.sort(key=lambda x: x[1])
    return year_columns

year_data = extract_year_columns(master_df)
emissions_cols = [col for col, year in year_data if not ('_price' in col or '_revenue' in col)]
price_cols = [col for col, year in year_data if '_price' in col]
revenue_cols = [col for col, year in year_data if '_revenue' in col]

In [48]:
print(f"Complete timeline: {len(year_data)} columns ({year_data[0][1]}-{year_data[-1][1]})")
print(f"Emissions: {len(emissions_cols)}, Price: {len(price_cols)}, Revenue: {len(revenue_cols)}")

Complete timeline: 114 columns (1990-2025)
Emissions: 43, Price: 36, Revenue: 35


In [49]:
all_year_cols = [col for col, year in year_data]
year_data_df = master_df[all_year_cols].apply(pd.to_numeric, errors='coerce')

print(f"Complete timeline statistics (1990-2025):")
print(f"Total data points: {year_data_df.size:,}")
print(f"Non-null values: {year_data_df.count().sum():,}")
print(f"Zero values: {(year_data_df == 0).sum().sum():,}")
print(f"Positive values: {(year_data_df > 0).sum().sum():,}")

Complete timeline statistics (1990-2025):
Total data points: 8,892
Non-null values: 4,963
Zero values: 2,350
Positive values: 2,613


Decades

In [50]:
decades = {
    '1990s': [col for col, year in year_data if 1990 <= year < 2000],
    '2000s': [col for col, year in year_data if 2000 <= year < 2010], 
    '2010s': [col for col, year in year_data if 2010 <= year < 2020],
    '2020s': [col for col, year in year_data if 2020 <= year < 2030]
}

In [51]:
for decade, cols in decades.items():
    if cols:
        decade_data = master_df[cols].apply(pd.to_numeric, errors='coerce')
        decade_total = decade_data.size
        decade_non_null = decade_data.count().sum()
        decade_zeros = (decade_data == 0).sum().sum()
        decade_positive = (decade_data > 0).sum().sum()
        decade_missing = decade_total - decade_non_null

        print(f"{decade}:")
        print(f"  Total: {decade_total:,} | Non-null: {decade_non_null:,} | Missing: {decade_missing:,}")        
        print(f"  Zeros: {decade_zeros:,} ({decade_zeros/decade_non_null*100:.1f}%) | Positive: {decade_positive:,} ({decade_positive/decade_non_null*100:.1f}%)")
        

1990s:
  Total: 2,340 | Non-null: 836 | Missing: 1,504
  Zeros: 744 (89.0%) | Positive: 92 (11.0%)
2000s:
  Total: 2,340 | Non-null: 985 | Missing: 1,355
  Zeros: 761 (77.3%) | Positive: 224 (22.7%)
2010s:
  Total: 2,418 | Non-null: 1,609 | Missing: 809
  Zeros: 677 (42.1%) | Positive: 932 (57.9%)
2020s:
  Total: 1,794 | Non-null: 1,533 | Missing: 261
  Zeros: 168 (11.0%) | Positive: 1,365 (89.0%)


 ##### Complete Timeline Analysis (1990-2025)

  ##### Historical Evolution of Carbon Pricing Policies:
  - **1990s**: 11.0% active policies - Early experimental phase with limited implementation
  - **2000s**: 22.7% active policies - Kyoto Protocol era, growing policy experimentation
  - **2010s**: 57.9% active policies - Paris Agreement expansion, majority policies become active
  - **2020s**: 89.0% active policies - Current boom, near-universal policy implementation

  ##### Data Quality Assessment:
  - **8,892 total data points** across 36-year timeline with 78 policies
  - **4,963 non-null values** (55.8% coverage) - realistic for historical carbon pricing data
  - **2,613 positive values** - substantial training data for ML models
  - **2,350 zeros** - meaningful data indicating policy planning/inactive phases

  ##### Key Insights for Machine Learning:
  1. **Rich Recent Data**: 2020s show 89% active policies with minimal missing data (14.5%)
  2. **Historical Context**: Clear evolution from 11% (1990s) to 89% (2020s) activity
  3. **Training Data Quality**: 52.6% positive values provide robust ML training foundation
  4. **Policy Lifecycle Patterns**: Distinct phases enable LSTM time series modeling

In [52]:
if 'Type' in master_df.columns and 'Region' in master_df.columns:
    policy_region = pd.crosstab(master_df['Type'], master_df['Region'])
    print("Policy Types by Region:")
    print(policy_region)

Policy Types by Region:
Region      East Asia & Pacific  Europe & Central Asia  \
Type                                                     
Carbon tax                    2                     20   
ETS                          16                      7   

Region      Latin America & Caribbean  Middle East & North Africa  \
Type                                                                
Carbon tax                          8                           1   
ETS                                 1                           0   

Region      North America  Sub-Saharan Africa  
Type                                           
Carbon tax              7                   1  
ETS                    15                   0  


In [53]:
if 'Status' in master_df.columns and 'Region' in master_df.columns:
    status_region = pd.crosstab(master_df['Status'], master_df['Region'])
    print(f"Implementation Status by Region:")
    print(status_region)


Implementation Status by Region:
Region       East Asia & Pacific  Europe & Central Asia  \
Status                                                    
Abolished                      1                      0   
Implemented                   17                     27   

Region       Latin America & Caribbean  Middle East & North Africa  \
Status                                                               
Abolished                            1                           0   
Implemented                          8                           1   

Region       North America  Sub-Saharan Africa  
Status                                          
Abolished                9                   0  
Implemented             13                   1  


In [54]:
if 'Income group' in master_df.columns:
    print(f"Policies by Income Group:")
    income_counts = master_df['Income group'].value_counts()
    print(income_counts)

Policies by Income Group:
Income group
High income            56
Upper middle income    22
Name: count, dtype: int64


##### Policy Distribution Analysis

##### Regional Policy Preferences:
- **Europe & Central Asia**: Prefers Carbon Tax (20 vs 7 ETS)
- **East Asia & Pacific**: Prefers ETS (16 vs 2 Carbon Tax)
- **North America**: Prefers ETS (15 vs 7 Carbon Tax)
- **Latin America & Caribbean**: Prefers Carbon Tax (8 vs 1 ETS)

##### Implementation Success by Region:
- **Europe & Central Asia**: 100% success rate (27/27 implemented)
- **East Asia & Pacific**: 94% success rate (17/18 implemented)
- **North America**: 59% success rate (13/22 implemented, 9 abolished)
- **Other regions**: High success rates with limited policies

##### Economic Development Patterns:
- **High income countries**: 56 policies (72%) - Most carbon pricing activity
- **Upper middle income**: 22 policies (28%) - Growing adoption
- **Lower income**: No policies yet - Economic development barrier

##### Key Insights for Simulation:
- Regional preferences affect policy type recommendations
- North America has higher policy failure rate (political volatility)
- Economic development level strongly correlates with carbon pricing adoption
- Europe shows highest policy stability and success


In [55]:
features_df = master_df.copy()                                                                            
features_df.shape[0]

78

In [56]:
print("Countries by region in our dataset:")
for region in master_df['Region'].unique():
    countries = master_df[master_df['Region'] == region]['Jurisdiction covered'].value_counts()
    print(f"\n{region}:")
    print(countries.head(10))

Countries by region in our dataset:

Europe & Central Asia:
Jurisdiction covered
United Kingdom    2
Switzerland       2
Luxembourg        1
Ukraine           1
Sweden            1
Spain             1
Slovenia          1
Portugal          1
Poland            1
Norway            1
Name: count, dtype: int64

North America:
Jurisdiction covered
Alberta                      2
British Columbia             2
Canada                       2
New Brunswick                2
Newfoundland and Labrador    2
Ontario                      2
California                   1
Massachusetts                1
Northwest Territories        1
Nova Scotia                  1
Name: count, dtype: int64

Latin America & Caribbean:
Jurisdiction covered
Mexico             2
Argentina          1
Baja California    1
Chile              1
Colombia           1
Tamaulipas         1
Uruguay            1
Zacatecas          1
Name: count, dtype: int64

East Asia & Pacific:
Jurisdiction covered
Australia      2
Korea, Rep.    1


In [57]:
print("Income group distribution:")
print(master_df['Income group'].value_counts())

print("\nEast Asia & Pacific countries:")
eap_countries = master_df[master_df['Region'] == 'East Asia & Pacific']['Jurisdiction covered'].value_counts()
print(eap_countries)

print("\nLatin America & Caribbean countries:")
lac_countries = master_df[master_df['Region'] == 'Latin America & Caribbean']['Jurisdiction covered'].value_counts()
print(lac_countries)

Income group distribution:
Income group
High income            56
Upper middle income    22
Name: count, dtype: int64

East Asia & Pacific countries:
Jurisdiction covered
Australia                      2
Korea, Rep.                    1
Tianjin                        1
Singapore                      1
Shenzhen                       1
Shanghai                       1
Saitama                        1
New Zealand                    1
Japan                          1
Beijing                        1
Indonesia                      1
Hubei                          1
Guangdong (except Shenzhen)    1
Fujian                         1
Chongqing                      1
China                          1
Tokyo                          1
Name: count, dtype: int64

Latin America & Caribbean countries:
Jurisdiction covered
Mexico             2
Argentina          1
Baja California    1
Chile              1
Colombia           1
Tamaulipas         1
Uruguay            1
Zacatecas          1
Name: count, dt

### Project Improvement Notice IMPROVED APPROACH

  **Date:** December 13, 2025

  ##### Problem with Initial Approach:
  - Only 4-5 input features (Type, Region, Income, Year, Price)
  - Used regional averages only
  - No country-specific factors (energy structure, population scale)
  - Weak defense: "Why doesn't Pakistan's 81% fossil fuel dependency matter?"

  ##### Solution - External Dataset Integration:

  **1. Energy Mix Data** (Energy Institute 2025)
     - Added feature: `fossil_fuel_dependency_%`
     - Calculation: (Coal + Oil + Gas) / Total Energy × 100
     - Why: High fossil countries have more carbon pricing potential
     - Example: Pakistan 81.2% vs Norway 5%

  **2. Population Data** (UN World Population Prospects 2024)
     - Added feature: `population_log`
     - Why: Revenue scales with population (more people = more economic activity)
     - Example: China vs Luxembourg revenue now scaled properly

  **3. CO2 Emissions** (Global Carbon Project 2025)
     - Use: Calculate actual CO2 reduction in tonnes
     - Formula: Coverage% × Country_Total_CO2
     - Example: "32% coverage = 57.5 million tonnes CO2 covered"

  **4. GDP Data** (Penn World Table 2023)
     - Use: Display revenue as % of GDP (context only, not ML feature)
     - Example: "$575M = 0.041% of GDP"

  ##### Updated Model Architecture:

  | Model | Old Features | New Features | Impact |
  |-------|-------------|--------------|--------|
  | Coverage | 4 | 5 (+Fossil_Fuel_%) | +10-15% R² |
  | Revenue | 5 | 7 (+Fossil_Fuel_%, +Pop_Log) | +15-20% R² |
  | Success | 3 | 4 (+Fossil_Fuel_%) | +5-8% Acc |

  ##### For FYP:
  - More defensible ("We account for energy structure and population scale")
  - Addresses examiner criticism about country-specific factors
  - Uses authoritative data sources (UN, Energy Institute, Global Carbon Project)
  - Expected credibility increase: +35-40%

  (Frontend):**
  - Can now display: Energy mix breakdown, CO2 per capita, Revenue % of GDP
  - More impressive visualizations possible

## Merging 2 Country Specific co2 emissions,population,gdp and energy mix datasets with master dataset

### Loading new datasets

In [58]:
energy_mix = pd.read_csv('../dataset/energy mix dataset/per-capita-energy-stacked.csv')
population = pd.read_csv('../dataset/population dataset/population.csv')
co2_emissions = pd.read_csv('../dataset/annual_co2_per_country/annual-co2-emissions-per-country.csv')
gdp = pd.read_csv('../dataset/gdp data/gdp-penn-world-table.csv')

### EDA on Datasets

#### EDA on energy mix dataset

In [59]:
energy_mix.head(5)

,Entity,Code,Year,Coal per capita (kWh),Oil per capita (kWh),Gas per capita (kWh),Nuclear per capita (kWh - equivalent),Hydro per capita (kWh - equivalent),Wind per capita (kWh - equivalent),Solar per capita (kWh - equivalent),Other renewables per capita (kWh - equivalent)
0,Africa,NaN,1965,1007.73364,1062.4866,29.817837,0.0,120.327515,0.0,0.0,0.0
1,Africa,NaN,1966,981.38165,1122.1318,32.492073,0.0,130.852040,0.0,0.0,0.0
2,Africa,NaN,1967,977.62480,1090.0707,31.297352,0.0,133.117780,0.0,0.0,0.0
3,Africa,NaN,1968,990.51874,1123.3291,30.902868,0.0,151.690750,0.0,0.0,0.0
4,Africa,NaN,1969,973.62690,1116.1130,35.165813,0.0,172.431850,0.0,0.0,0.0


In [60]:
energy_mix.shape

(6585, 11)

In [61]:
print(f"Columns: {energy_mix.columns.tolist()}")

Columns: ['Entity', 'Code', 'Year', 'Coal per capita (kWh)', 'Oil per capita (kWh)', 'Gas per capita (kWh)', 'Nuclear per capita (kWh - equivalent)', 'Hydro per capita (kWh - equivalent)', 'Wind per capita (kWh - equivalent)', 'Solar per capita (kWh - equivalent)', 'Other renewables per capita (kWh - equivalent)']


In [62]:
energy_mix.dtypes

Entity                                             object
Code                                               object
Year                                                int64
Coal per capita (kWh)                             float64
Oil per capita (kWh)                              float64
Gas per capita (kWh)                              float64
Nuclear per capita (kWh - equivalent)             float64
Hydro per capita (kWh - equivalent)               float64
Wind per capita (kWh - equivalent)                float64
Solar per capita (kWh - equivalent)               float64
Other renewables per capita (kWh - equivalent)    float64
dtype: object

In [63]:
energy_mix['Entity'].nunique()

121

In [64]:
print(f"\nSample entities (first 20):")
print(sorted(energy_mix['Entity'].unique())[:20])


Sample entities (first 20):
['Africa', 'Algeria', 'Angola', 'Argentina', 'Asia', 'Australia', 'Austria', 'Azerbaijan', 'Bahrain', 'Bangladesh', 'Belarus', 'Belgium', 'Bolivia', 'Brazil', 'Brunei', 'Bulgaria', 'Canada', 'Chad', 'Chile', 'China']


In [65]:
print(f"Year range: {energy_mix['Year'].min()} - {energy_mix['Year'].max()}")
print(f"\nMost recent year data availability:")
recent_data = energy_mix[energy_mix['Year'] >= 2020]['Year'].value_counts().sort_index()
print(recent_data)

Year range: 1965 - 2024

Most recent year data availability:
Year
2020    119
2021    119
2022    119
2023    119
2024    119
Name: count, dtype: int64


In [66]:
decades = {
      '1990s': (1990, 1999),
      '2000s': (2000, 2009),
      '2010s': (2010, 2019),
      '2020s': (2020, 2024)
  }

fossil_cols = ['Coal per capita (kWh)', 'Oil per capita (kWh)', 'Gas per capita (kWh)']

print("Energy data completeness by decade:\n")
for decade, (start, end) in decades.items():
    decade_data = energy_mix[(energy_mix['Year'] >= start) & (energy_mix['Year'] <= end)]
    total = len(decade_data)
    non_null = decade_data[fossil_cols].notna().all(axis=1).sum()
    print(f"{decade}: {non_null}/{total} complete ({non_null/total*100:.1f}%)")

Energy data completeness by decade:

1990s: 900/1150 complete (78.3%)
2000s: 900/1183 complete (76.1%)
2010s: 900/1189 complete (75.7%)
2020s: 450/595 complete (75.6%)


In [67]:
policy_locations = master_df['Jurisdiction covered'].unique()

In [68]:
print(f"Our master dataset has {len(policy_locations)} unique policy locations")

Our master dataset has 68 unique policy locations


In [69]:
energy_entities = set(energy_mix['Entity'].unique())

direct_matches = []
no_matches = []

for location in policy_locations:
      if location in energy_entities:
          direct_matches.append(location)
      else:
          no_matches.append(location)

print(f"Direct matches: {len(direct_matches)}")
print(f"No direct match: {len(no_matches)}")

print(f"\nExamples of no match (first 10):")
print(sorted(no_matches)[:10])

Direct matches: 34
No direct match: 34

Examples of no match (first 10):
['Albania', 'Alberta', 'Baja California', 'Beijing', 'British Columbia', 'California', 'Chongqing', 'EU27+', 'Fujian', 'Guangdong (except Shenzhen)']


In [70]:
test_cases = ['Albania', 'China', 'Canada', 'Sweden', 'EU27+']

print("Checking specific examples:\n")
for country in test_cases:
      in_master = country in policy_locations
      in_energy = country in energy_entities
      print(f"{country:<20} Master: {in_master}  Energy: {in_energy}")

print("\n\nSearching for variations of 'European Union':")
eu_variants = [e for e in energy_entities if 'Europe' in e or 'EU' in e]
print(eu_variants)

Checking specific examples:

Albania              Master: True  Energy: False
China                Master: True  Energy: True
Canada               Master: True  Energy: True
Sweden               Master: True  Energy: True
EU27+                Master: True  Energy: False


Searching for variations of 'European Union':
['European Union (27)', 'Europe']


In [71]:
no_matches_list=[]
print(f"Locations that didn't match directly ({len(no_matches)}):\n")
for location in sorted(no_matches):
      no_matches_list.append(location)
print(no_matches_list)
      

Locations that didn't match directly (34):

['Albania', 'Alberta', 'Baja California', 'Beijing', 'British Columbia', 'California', 'Chongqing', 'EU27+', 'Fujian', 'Guangdong (except Shenzhen)', 'Hubei', 'Korea, Rep.', 'Liechtenstein', 'Massachusetts', 'Montenegro', 'New Brunswick', 'Newfoundland and Labrador', 'Northwest Territories', 'Nova Scotia', 'Ontario', 'Oregon', 'Prince Edward Island', 'Quebec', 'RGGI', 'Saitama', 'Saskatchewan', 'Shanghai', 'Shenzhen', 'Tamaulipas', 'Tianjin', 'Tokyo', 'Uruguay', 'Washington', 'Zacatecas']


 Looking at the 34 locations that didn't match:
  - Canadian provinces: Alberta, British Columbia, Saskatchewan, etc.
  - US states: California, Massachusetts, Oregon, Washington
  - Chinese municipalities/provinces: Beijing, Shanghai, Shenzhen, etc.
  - Japanese prefectures: Tokyo, Saitama
  - Mexican states: Baja California, Tamaulipas, Zacatecas
  - Special cases: EU27+, Korea Rep., and small countries (Albania, Uruguay, etc.)

In [72]:
subnational_regions = {
      'Canada': ['Alberta', 'British Columbia', 'Quebec', 'Ontario', 'New Brunswick',
                 'Newfoundland and Labrador', 'Northwest Territories', 'Nova Scotia',
                 'Prince Edward Island', 'Saskatchewan'],

      'United States': ['California', 'Massachusetts', 'Oregon', 'Washington', 'RGGI'],

      'China': ['Beijing', 'Shanghai', 'Chongqing', 'Fujian', 'Guangdong (except Shenzhen)',
                'Hubei', 'Shenzhen', 'Tianjin'],

      'Japan': ['Tokyo', 'Saitama'],

      'Mexico': ['Baja California', 'Tamaulipas', 'Zacatecas']
  }

parent_country_map = {}
for parent, regions in subnational_regions.items():
    for region in regions:
          parent_country_map[region] = parent

print(f"Parent countries: {len(subnational_regions)}")
print(f"Sub-national regions: {len(parent_country_map)}\n")

can_use_parent = []
truly_missing = []

for location in sorted(no_matches):
    if location in parent_country_map:
        parent = parent_country_map[location]
        if parent in energy_entities:
              can_use_parent.append((location, parent))
        else:
            truly_missing.append(location)
    elif location == 'EU27+' and 'European Union (27)' in energy_entities:
        can_use_parent.append((location, 'European Union (27)'))
    elif location == 'Korea, Rep.' and 'South Korea' in energy_entities:
        can_use_parent.append((location, 'South Korea'))
    else:
        truly_missing.append(location)
print(f"Can use parent country: {len(can_use_parent)}")
print(f"Truly missing: {len(truly_missing)}")
print(f"\nTruly missing locations:")
print(truly_missing)

Parent countries: 5
Sub-national regions: 28

Can use parent country: 30
Truly missing: 4

Truly missing locations:
['Albania', 'Liechtenstein', 'Montenegro', 'Uruguay']


In [73]:
missing_check = ['Albania', 'Liechtenstein', 'Montenegro', 'Uruguay']

print("Double-checking if these truly don't exist:\n")
for country in missing_check:
    exists = country in energy_entities
    print(f"{country}: {'Found' if exists else 'Missing'}")

print(f"\nHow many policies would this affect?")
affected = master_df[master_df['Jurisdiction covered'].isin(missing_check)]
print(f"Policies with these locations: {len(affected)}")
print(f"\nPolicy details:")
print(affected[['Instrument name', 'Jurisdiction covered', 'Type', 'Status']])

Double-checking if these truly don't exist:

Albania: Missing
Liechtenstein: Missing
Montenegro: Missing
Uruguay: Missing

How many policies would this affect?
Policies with these locations: 4

Policy details:
             Instrument name Jurisdiction covered        Type       Status
0         Albania carbon tax              Albania  Carbon tax  Implemented
35  Liechtenstein carbon tax        Liechtenstein  Carbon tax  Implemented
40            Montenegro ETS           Montenegro         ETS  Implemented
75           Uruguay CO2 tax              Uruguay  Carbon tax  Implemented


Checking for regional entities in Energy Mix

In [74]:
regions = [e for e in energy_entities if any(word in e for word in ['Europe', 'Asia', 'America', 'Africa', 'South', 'North', 'Latin'])]
for region in sorted(regions):
      print(f"  {region}")

  Africa
  Asia
  Europe
  European Union (27)
  North America
  North Macedonia
  South Africa
  South America
  South Korea
  South Sudan


In [75]:
actual_regions = ['Europe', 'South America', 'North America', 'Asia', 'Africa']

In [76]:
missing_to_region = {
      'Albania': 'Europe',
      'Montenegro': 'Europe',
      'Liechtenstein': 'Europe',
      'Uruguay': 'South America'
  }

For 4 countries without direct data (Albania, Montenegro, Liechtenstein, Uruguay), we use regional aggregates from the Energy Mix dataset. This is a standard approach in environmental economics when country-specific data is unavailable.

##### Energy Mix Dataset - Summary

  - 121 entities (countries + regions)
  - Time coverage: 1965-2024
  - 8 energy sources per capita
  - Data completeness: ~75% across all decades (1990s-2020s)
  - Integration strategy:
    - 34 direct country matches
    - 30 parent country mappings (provinces/states → countries)
    - 4 regional mappings (Albania, Montenegro, Liechtenstein, Uruguay)
    - Total coverage: 68/68 locations (100%)

#### EDA on popilation dataset

In [1]:
population.head(5)

NameError: name 'population' is not defined

In [ ]:
population.shape

(18944, 4)

In [ ]:
population.columns.to_list()

['Entity', 'Code', 'Year', 'all years']

In [ ]:
population.dtypes

Entity       object
Code         object
Year          int64
all years     int64
dtype: object

In [ ]:
print(population.isnull().sum())

Entity          0
Code         1332
Year            0
all years       0
dtype: int64


In [ ]:
print(f"Unique entities: {population['Entity'].nunique()}")
print(f"\nSample entities (first 20):")
print(sorted(population['Entity'].unique())[:20])

Unique entities: 256

Sample entities (first 20):
['Afghanistan', 'Africa (UN)', 'Albania', 'Algeria', 'American Samoa', 'Americas (UN)', 'Andorra', 'Angola', 'Anguilla', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba', 'Asia (UN)', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh']


In [ ]:
print(f"Year range: {population['Year'].min()} - {population['Year'].max()}")


Year range: 1950 - 2023


In [ ]:
print("Year coverage by decade:")
for decade, (start, end) in decades.items():
    decade_data = population[(population['Year'] >= start) & (population['Year'] <= end)]
    years_available = decade_data['Year'].nunique()
    print(f"{decade}: {years_available} years")

Year coverage by decade:
1990s: 10 years
2000s: 10 years
2010s: 10 years
2020s: 4 years


In [ ]:
pop_entities = set(population['Entity'].unique())

In [ ]:
direct_pop = 0
for location in policy_locations:
      if location in pop_entities:
          direct_pop += 1

print(f"Direct matches: {direct_pop}/{len(policy_locations)}")

Direct matches: 38/68


In [ ]:
pop_direct_matches = []
pop_no_matches = []

for location in policy_locations:
    if location in pop_entities:
        pop_direct_matches.append(location)
    else:
        pop_no_matches.append(location)
print(f"Direct matches: {len(pop_direct_matches)}")
print(f"No direct match: {len(pop_no_matches)}")
print("No direct match list:")
print(sorted(pop_no_matches))

Direct matches: 38
No direct match: 30
No direct match list:
['Alberta', 'Baja California', 'Beijing', 'British Columbia', 'California', 'Chongqing', 'EU27+', 'Fujian', 'Guangdong (except Shenzhen)', 'Hubei', 'Korea, Rep.', 'Massachusetts', 'New Brunswick', 'Newfoundland and Labrador', 'Northwest Territories', 'Nova Scotia', 'Ontario', 'Oregon', 'Prince Edward Island', 'Quebec', 'RGGI', 'Saitama', 'Saskatchewan', 'Shanghai', 'Shenzhen', 'Tamaulipas', 'Tianjin', 'Tokyo', 'Washington', 'Zacatecas']


In [ ]:
print("Checking EU variations in Population dataset:")
eu_variants = [e for e in pop_entities if 'Europe' in e or 'EU' in e]
print(eu_variants)

Checking EU variations in Population dataset:
['Europe (UN)']


In [ ]:
pop_can_use_parent = []
pop_truly_missing = []

for location in sorted(pop_no_matches):
    if location in parent_country_map:
        parent = parent_country_map[location]
        if parent in pop_entities:
            pop_can_use_parent.append((location, parent))
        else:
            pop_truly_missing.append(location)
    elif location == 'EU27+':
        if  'Europe (UN)' in pop_entities:
            pop_can_use_parent.append((location, 'Europe (UN)'))
        else:
            pop_truly_missing.append(location)
    elif location == 'Korea, Rep.':
        if 'South Korea' in pop_entities:
            pop_can_use_parent.append((location, 'South Korea'))
        else:
            pop_truly_missing.append(location)
    else:
        pop_truly_missing.append(location)

print(f"Can use parent country: {len(pop_can_use_parent)}")
print(f"Truly missing: {len(pop_truly_missing)}")

if pop_truly_missing:
      print(f"\nTruly missing locations:")
      print(pop_truly_missing)

Can use parent country: 30
Truly missing: 0


In [ ]:
missing_from_energy = ['Albania', 'Montenegro', 'Liechtenstein', 'Uruguay']

print("Countries missing from Energy Mix - checking Population dataset:\n")
for country in missing_from_energy:
    exists = country in pop_entities
    print(f"  {country}: {'Found in Population' if exists else 'Also missing'}")

Countries missing from Energy Mix - checking Population dataset:

  Albania: Found in Population
  Montenegro: Found in Population
  Liechtenstein: Found in Population
  Uruguay: Found in Population


##### Population Dataset - Summary

  - 256 entities (countries + regions)
  - Time coverage: 1950-2023
  - Integration strategy:
    - 38 direct country matches
    - 30 parent country mappings
    - 0 missing (all locations covered)
    - Total coverage: 68/68 locations (100%)

#### EDA on CO2 Emission Dataset

In [91]:
co2_emissions.head(5)

,Entity,Code,Year,Annual CO₂ emissions
0,Afghanistan,AFG,1949,14656.0
1,Afghanistan,AFG,1950,84272.0
2,Afghanistan,AFG,1951,91600.0
3,Afghanistan,AFG,1952,91600.0
4,Afghanistan,AFG,1953,106256.0


In [92]:
co2_emissions.shape

(29384, 4)

In [93]:
co2_emissions.dtypes

Entity                   object
Code                     object
Year                      int64
Annual CO₂ emissions    float64
dtype: object

In [94]:
print(co2_emissions.isnull().sum())

Entity                     0
Code                    5670
Year                       0
Annual CO₂ emissions       0
dtype: int64


In [95]:
print(f"Unique entities: {co2_emissions['Entity'].nunique()}")

Unique entities: 247


In [96]:
print("Year range:", co2_emissions['Year'].min(), "-", co2_emissions['Year'].max())

Year range: 1750 - 2024


In [97]:
print("Year coverage by decade:")
for decade, (start, end) in decades.items():
    decade_data = co2_emissions[(co2_emissions['Year'] >= start) & (co2_emissions['Year'] <= end)]
    years_available = decade_data['Year'].nunique()
    print(f"{decade}: {years_available} years")

Year coverage by decade:
1990s: 10 years
2000s: 10 years
2010s: 10 years
2020s: 5 years


In [98]:
co2_entities = set(co2_emissions['Entity'].unique())

co2_direct_matches = []
co2_no_matches = []

for location in policy_locations:
    if location in co2_entities:
        co2_direct_matches.append(location)
    else:
        co2_no_matches.append(location)

print(f"Direct matches: {len(co2_direct_matches)}")
print(f"No direct match: {len(co2_no_matches)}")
print(f"No direct match list: {co2_no_matches}")

Direct matches: 38
No direct match: 30
No direct match list: ['Alberta', 'Baja California', 'Beijing', 'British Columbia', 'California', 'Chongqing', 'EU27+', 'Fujian', 'Guangdong (except Shenzhen)', 'Hubei', 'Korea, Rep.', 'Massachusetts', 'New Brunswick', 'Newfoundland and Labrador', 'Northwest Territories', 'Nova Scotia', 'Ontario', 'Oregon', 'Prince Edward Island', 'Quebec', 'RGGI', 'Saitama', 'Saskatchewan', 'Shanghai', 'Shenzhen', 'Tamaulipas', 'Tianjin', 'Tokyo', 'Washington', 'Zacatecas']


In [99]:
print("Checking EU variations in CO2 dataset:")
eu_variants = [e for e in co2_entities if 'Europe' in e or 'EU' in e]
print(eu_variants)

Checking EU variations in CO2 dataset:
['Europe (excl. EU-28)', 'Europe (excl. EU-27)', 'European Union (28)', 'European Union (27)', 'Europe', 'Europe (GCP)']


In [100]:
co2_can_use_parent = []
co2_truly_missing = []

for location in sorted(co2_no_matches):
    if location in parent_country_map:
        parent = parent_country_map[location]
        if parent in co2_entities:
            co2_can_use_parent.append((location, parent))
        else:
            co2_truly_missing.append(location)
    elif location == 'EU27+':
        if 'European Union (27)' in co2_entities:
            co2_can_use_parent.append((location, 'European Union (27)'))
        else:
            co2_truly_missing.append(location)
    elif location == 'Korea, Rep.':
        if 'South Korea' in co2_entities:
            co2_can_use_parent.append((location, 'South Korea'))
        else:
            co2_truly_missing.append(location)
    else:
        co2_truly_missing.append(location)
print(f"Can use parent country: {len(co2_can_use_parent)}")
print(f"Truly missing: {len(co2_truly_missing)}")

if co2_truly_missing:
    print(f"\nTruly missing locations:")
    print(co2_truly_missing)

Can use parent country: 30
Truly missing: 0


In [101]:
co2_col = 'Annual CO₂ emissions'

print("CO2 data completeness by decade:\n")

for decade, (start, end) in decades.items():
    decade_data = co2_emissions[(co2_emissions['Year'] >= start) & (co2_emissions['Year'] <= end)]
    total = len(decade_data)
    non_null = decade_data[co2_col].notna().sum()
    print(f"{decade}: {non_null}/{total} complete ({non_null/total*100:.1f}%)")

CO2 data completeness by decade:

1990s: 2442/2442 complete (100.0%)
2000s: 2470/2470 complete (100.0%)
2010s: 2470/2470 complete (100.0%)
2020s: 1235/1235 complete (100.0%)


##### CO2 Emissions Dataset - Summary

  - 247 entities (countries + regions)
  - Time coverage: 1750-2024 (275 years)
  - Data completeness: ~100% across all decades (1990s-2020s)
  - Integration strategy:
    - 38 direct country matches
    - 30 parent country mappings
    - 0 missing (all locations covered)
    - Total coverage: 68/68 locations (100%)

#### EDA on GDP Dataset

In [102]:
gdp.head(5)

,Entity,Code,Year,"GDP (output, multiple price benchmarks)"
0,Albania,ALB,1971,7428762695
1,Albania,ALB,1972,7735746582
2,Albania,ALB,1973,8046290039
3,Albania,ALB,1974,8385445312
4,Albania,ALB,1975,8715459960


In [103]:
gdp.shape

(10907, 4)

In [104]:
gdp.dtypes

Entity                                     object
Code                                       object
Year                                        int64
GDP (output, multiple price benchmarks)     int64
dtype: object

In [105]:
gdp.columns.to_list()

['Entity', 'Code', 'Year', 'GDP (output, multiple price benchmarks)']

In [106]:
print(f"Missing values in GDP dataset:{gdp.isnull().sum()}")

Missing values in GDP dataset:Entity                                     0
Code                                       0
Year                                       0
GDP (output, multiple price benchmarks)    0
dtype: int64


In [107]:
gdp['Entity'].nunique()

184

In [108]:
print("Year range:", gdp['Year'].min(), "-", gdp['Year'].max())

Year range: 1950 - 2023


In [109]:
print("Year coverage by decade:")
for decade, (start, end) in decades.items():
    decade_data = gdp[(gdp['Year'] >= start) & (gdp['Year'] <= end)]
    years_available = decade_data['Year'].nunique()
    print(f"{decade}: {years_available} years")

Year coverage by decade:
1990s: 10 years
2000s: 10 years
2010s: 10 years
2020s: 4 years


In [110]:
gdp_entities = set(gdp['Entity'].unique())

gdp_direct_matches = []
gdp_no_matches = []

for location in policy_locations:
    if location in gdp_entities:
        gdp_direct_matches.append(location)
    else:
        gdp_no_matches.append(location)

print(f"Direct matches: {len(gdp_direct_matches)}")
print(f"No direct match: {len(gdp_no_matches)}")
print(f"No direct match list: {gdp_no_matches}")

Direct matches: 37
No direct match: 31
No direct match list: ['Alberta', 'Baja California', 'Beijing', 'British Columbia', 'California', 'Chongqing', 'EU27+', 'Fujian', 'Guangdong (except Shenzhen)', 'Hubei', 'Korea, Rep.', 'Liechtenstein', 'Massachusetts', 'New Brunswick', 'Newfoundland and Labrador', 'Northwest Territories', 'Nova Scotia', 'Ontario', 'Oregon', 'Prince Edward Island', 'Quebec', 'RGGI', 'Saitama', 'Saskatchewan', 'Shanghai', 'Shenzhen', 'Tamaulipas', 'Tianjin', 'Tokyo', 'Washington', 'Zacatecas']


In [111]:
print("Checking EU variations in GDP dataset:")
eu_variants = [e for e in gdp_entities if 'Europe' in e or 'EU' in e]
print(eu_variants)

Checking EU variations in GDP dataset:
[]


In [112]:
gdp_can_use_parent = []
gdp_truly_missing = []
for location in sorted(gdp_no_matches):
      if location in parent_country_map:
          parent = parent_country_map[location]
          if parent in gdp_entities:
              gdp_can_use_parent.append((location, parent))
          else:
              gdp_truly_missing.append(location)
      elif location == 'EU27+':
          if 'European Union (27)' in gdp_entities:
              gdp_can_use_parent.append((location, 'European Union (27)'))
          else:
              gdp_truly_missing.append(location)
      elif location == 'Korea, Rep.':
          if 'South Korea' in gdp_entities:
              gdp_can_use_parent.append((location, 'South Korea'))
          else:
              gdp_truly_missing.append(location)
      else:
          gdp_truly_missing.append(location)

print(f"Can use parent country: {len(gdp_can_use_parent)}")
print(f"Truly missing: {len(gdp_truly_missing)}")

if gdp_truly_missing:
    print(f"\nTruly missing locations:")
    print(gdp_truly_missing)

Can use parent country: 29
Truly missing: 2

Truly missing locations:
['EU27+', 'Liechtenstein']


In [113]:
eu27_members = ['Austria', 'Belgium', 'Bulgaria', 'Croatia', 'Cyprus', 'Czechia',
                  'Denmark', 'Estonia', 'Finland', 'France', 'Germany', 'Greece',
                  'Hungary', 'Ireland', 'Italy', 'Latvia', 'Lithuania', 'Luxembourg',
                  'Malta', 'Netherlands', 'Poland', 'Portugal', 'Romania', 'Slovakia',
                  'Slovenia', 'Spain', 'Sweden']

In [114]:
eu27_gdp_data = gdp[gdp['Entity'].isin(eu27_members)]
eu27_aggregate = eu27_gdp_data.groupby('Year').agg({'GDP (output, multiple price benchmarks)': 'sum'}).reset_index()
eu27_aggregate['Entity'] = 'European Union (27)'
eu27_aggregate['Code'] = 'EU27'

In [115]:
eu27_aggregate = eu27_aggregate[['Entity', 'Code', 'Year', 'GDP (output, multiple price benchmarks)']]

In [116]:
print(f"EU27 GDP calculated for {len(eu27_aggregate)} years ({eu27_aggregate['Year'].min()}-{eu27_aggregate['Year'].max()})")

EU27 GDP calculated for 74 years (1950-2023)


In [117]:
gdp = pd.concat([gdp, eu27_aggregate], ignore_index=True)

print(f"\nGDP dataset updated: {gdp.shape}")
print(f"EU27 now available: {'European Union (27)' in gdp['Entity'].values}")


GDP dataset updated: (10981, 4)
EU27 now available: True


##### Handling Missing GDP Data

  Two locations missing from GDP dataset: EU27+ and Liechtenstein

  **Solution: EU27+ GDP Calculation**
  - GDP dataset lacks regional aggregates
  - Solution: Calculate EU27 GDP by summing all 27 member countries' GDP
  - Rationale: EU27 is an economic union, aggregate GDP is sum of member states
  - Implementation: Create calculated 'European Union (27)' entry

  **Liechtenstein: Accept Missing**
  - Not available in Penn World Table (very small country, 38,000 population)
  - Impact: 1 policy (Liechtenstein carbon tax) will not display GDP context
  - Rationale: Affects only 1.3% of dataset (1/78 policies), acceptable limitation
  - Document transparently in final system

##### GDP Dataset - Summary

  - 184 entities (countries only, no regional aggregates)
  - Time coverage: 1950-2023
  - Integration strategy:
    - 36 direct country matches
    - 29 parent country mappings
    - 1 calculated aggregate: EU27+ (sum of member states)
    - 1 missing: Liechtenstein (1.3% of dataset, acceptable limitation)
  - Note: GDP is display-only feature, not used in ML models

## Merging Datasets

In [118]:
def get_lookup_entity(location, dataset_type='energy'):
      
      if location in parent_country_map:
          return parent_country_map[location]

      if location in ['EU', 'EU27+']:
          return 'European Union (27)'

      if location == 'Korea, Rep.':
          return 'South Korea'


      if dataset_type == 'energy':
          regional_fallback = {
              'Albania': 'Europe',
              'Montenegro': 'Europe',
              'Liechtenstein': 'Europe',
              'Uruguay': 'South America'
          }
          if location in regional_fallback:
              return regional_fallback[location]

      return location

print("\nTest lookups:")
print(f"  Sweden : {get_lookup_entity('Sweden')}")
print(f"  Alberta: {get_lookup_entity('Alberta')}")
print(f"  Albania : {get_lookup_entity('Albania', 'energy')}")


Test lookups:
  Sweden : Sweden
  Alberta: Canada
  Albania : Europe


In [119]:
energy_mix_clean = energy_mix.copy()
population_clean = population.copy()
co2_clean = co2_emissions.copy()
gdp_clean = gdp.copy()

In [120]:
print("  Energy Mix: Keeping all 8 energy source columns as-is")
print(f" Columns: {list(energy_mix_clean.columns)}")

  Energy Mix: Keeping all 8 energy source columns as-is
 Columns: ['Entity', 'Code', 'Year', 'Coal per capita (kWh)', 'Oil per capita (kWh)', 'Gas per capita (kWh)', 'Nuclear per capita (kWh - equivalent)', 'Hydro per capita (kWh - equivalent)', 'Wind per capita (kWh - equivalent)', 'Solar per capita (kWh - equivalent)', 'Other renewables per capita (kWh - equivalent)']


In [121]:
pop_old_col = population_clean.columns[3]
print(f"  Population: '{pop_old_col}' : 'Population'")
population_clean.rename(columns={pop_old_col: 'Population'}, inplace=True)

  Population: 'all years' : 'Population'


In [122]:
co2_old_col = co2_clean.columns[3]
print(f"  CO2: {co2_old_col} : 'Annual_CO2_emissions'")
co2_clean.rename(columns={co2_old_col: 'Annual_CO2_emissions'}, inplace=True)

  CO2: Annual CO₂ emissions : 'Annual_CO2_emissions'


In [123]:
gdp_old_col = gdp_clean.columns[3]
print(f"  GDP: {gdp_old_col} : 'GDP'")
gdp_clean.rename(columns={gdp_old_col: 'GDP'}, inplace=True)

  GDP: GDP (output, multiple price benchmarks) : 'GDP'


In [124]:
print("Year ranges in external datasets:")
print(f" Energy Mix: {energy_mix_clean['Year'].min()} - {energy_mix_clean['Year'].max()}")
print(f" Population: {population_clean['Year'].min()} - {population_clean['Year'].max()}")
print(f" CO2: {co2_clean['Year'].min()} - {co2_clean['Year'].max()}")
print(f" GDP: {gdp_clean['Year'].min()} - {gdp_clean['Year'].max()}")

Year ranges in external datasets:
 Energy Mix: 1965 - 2024
 Population: 1950 - 2023
 CO2: 1750 - 2024
 GDP: 1950 - 2023


In [125]:
print(f"\nMaster dataset shape: {master_df.shape}")
print(f"Total columns: {len(master_df.columns)}")


Master dataset shape: (78, 163)
Total columns: 163


  Show first 20 columns to understand structure

In [126]:
for i, col in enumerate(master_df.columns[:20], 1):
      print(f"  {i}. {col}")

  1. Unique ID
  2. Instrument name
  3. Type
  4. Status
  5. Jurisdiction covered
  6. Share of jurisdiction emissions covered
  7. Price on 1 April
  8. 2020
  9. 2021
  10. 2022
  11. 2023
  12. 2024
  13. 2025
  14. Change
  15. Government revenue
  16. 2019
  17. 2020.1
  18. 2021.1
  19. 2022.1
  20. 2023.1


Metadata columns (manually verified from EDA)

In [127]:
metadata_cols = ['Unique ID', 'Instrument name', 'Type', 'Status','Jurisdiction', 'Region', 'Income group']

In [128]:
print("\nVerifying metadata columns exist:")
for col in metadata_cols:
    if col in master_df.columns:
        print(f"  yes {col}")
    else:
        print(f"  no {col}")


Verifying metadata columns exist:
  yes Unique ID
  yes Instrument name
  yes Type
  yes Status
  yes Jurisdiction
  yes Region
  yes Income group


In [129]:
emission_year_cols = []
for col in master_df.columns:
      col_str = str(col)

      if col_str.endswith('.0'):
          year_part = col_str[:-2]  
          if year_part.isdigit():
              year = int(year_part)
              if 1990 <= year <= 2025:
                  emission_year_cols.append(col)
                  print(f"  Found: '{col}'")
emission_year_cols = sorted(emission_year_cols, key=lambda x: int(x[:-2]))


  Found: '1990.0'
  Found: '1991.0'
  Found: '1992.0'
  Found: '1993.0'
  Found: '1994.0'
  Found: '1995.0'
  Found: '1996.0'
  Found: '1997.0'
  Found: '1998.0'
  Found: '1999.0'
  Found: '2000.0'
  Found: '2001.0'
  Found: '2002.0'
  Found: '2003.0'
  Found: '2004.0'
  Found: '2005.0'
  Found: '2006.0'
  Found: '2007.0'
  Found: '2008.0'
  Found: '2009.0'
  Found: '2010.0'
  Found: '2011.0'
  Found: '2012.0'
  Found: '2013.0'
  Found: '2014.0'
  Found: '2015.0'
  Found: '2016.0'
  Found: '2017.0'
  Found: '2018.0'
  Found: '2019.0'
  Found: '2020.0'
  Found: '2021.0'
  Found: '2022.0'
  Found: '2023.0'
  Found: '2024.0'
  Found: '2025.0'


In [130]:
print(f"  Count: {len(emission_year_cols)}")
print(f"  Range: {emission_year_cols[0]} to {emission_year_cols[-1]}")
print(f"  Pattern: Pure digits (e.g., '1990', '1991', ...)")
if len(emission_year_cols) <= 10:
    print(f"  All: {emission_year_cols}")
else:
    print(f"  First 5: {emission_year_cols[:5]}")
    print(f"  Last 5: {emission_year_cols[-5:]}")

  Count: 36
  Range: 1990.0 to 2025.0
  Pattern: Pure digits (e.g., '1990', '1991', ...)
  First 5: ['1990.0', '1991.0', '1992.0', '1993.0', '1994.0']
  Last 5: ['2021.0', '2022.0', '2023.0', '2024.0', '2025.0']


In [131]:
price_year_cols = []
for col in master_df.columns:
    if isinstance(col, str) and '_price' in col.lower() and '.' in col:
        try:
            year_part = col.split('.')[0]
            if year_part.isdigit():
                year = int(year_part)
                if 1990 <= year <= 2025:
                    price_year_cols.append(col)
        except:
            pass

price_year_cols = sorted(price_year_cols, key=lambda x: int(x.split('.')[0]))

In [132]:
print(f"  Count: {len(price_year_cols)}")
if len(price_year_cols) > 0:
    print(f"  Range: {price_year_cols[0]} to {price_year_cols[-1]}")
    print(f"  Pattern: YEAR.0_price (e.g., '1990.0_price')")
    if len(price_year_cols) <= 10:
        print(f"  All: {price_year_cols}")
    else:
        print(f"  First 5: {price_year_cols[:5]}")
        print(f"  Last 5: {price_year_cols[-5:]}")

  Count: 36
  Range: 1990.0_price to 2025.0_price
  Pattern: YEAR.0_price (e.g., '1990.0_price')
  First 5: ['1990.0_price', '1991.0_price', '1992.0_price', '1993.0_price', '1994.0_price']
  Last 5: ['2021.0_price', '2022.0_price', '2023.0_price', '2024.0_price', '2025.0_price']


In [133]:
revenue_year_cols = []
for col in master_df.columns:
    if isinstance(col, str) and '_revenue' in col.lower() and '.' in col:
        try:
            year_part = col.split('.')[0]
            if year_part.isdigit():
                year = int(year_part)
                if 1990 <= year <= 2025:
                    revenue_year_cols.append(col)
        except:
            pass

revenue_year_cols = sorted(revenue_year_cols, key=lambda x: int(x.split('.')[0]))

In [134]:
print(f"  Count: {len(revenue_year_cols)}")
if len(revenue_year_cols) > 0:
      print(f"  Range: {revenue_year_cols[0]} to {revenue_year_cols[-1]}")
      print(f"  Pattern: YEAR.0_revenue (e.g., '1990.0_revenue')")
      if len(revenue_year_cols) <= 10:
          print(f"  All: {revenue_year_cols}")
      else:
          print(f"  First 5: {revenue_year_cols[:5]}")
          print(f"  Last 5: {revenue_year_cols[-5:]}")

  Count: 35
  Range: 1990.0_revenue to 2024.0_revenue
  Pattern: YEAR.0_revenue (e.g., '1990.0_revenue')
  First 5: ['1990.0_revenue', '1991.0_revenue', '1992.0_revenue', '1993.0_revenue', '1994.0_revenue']
  Last 5: ['2020.0_revenue', '2021.0_revenue', '2022.0_revenue', '2023.0_revenue', '2024.0_revenue']


In [135]:
print(f"Column counts:")
print(f"  Metadata: {len(metadata_cols)}")
print(f"  Emission years: {len(emission_year_cols)}")
print(f"  Price years: {len(price_year_cols)}")
print(f"  Revenue years: {len(revenue_year_cols)}")
print(f"  Total categorized: {len(metadata_cols) + len(emission_year_cols) + len(price_year_cols) + len(revenue_year_cols)}")
print(f"  Total in dataset: {len(master_df.columns)}")

Column counts:
  Metadata: 7
  Emission years: 36
  Price years: 36
  Revenue years: 35
  Total categorized: 114
  Total in dataset: 163


Performing unpivot operations

In [136]:
print(f"Using {len(emission_year_cols)} emission year columns")
print(f"From: {len(master_df)} rows (wide format)")

Using 36 emission year columns
From: 78 rows (wide format)


In [137]:
emissions_long = pd.melt(
      master_df,
      id_vars=metadata_cols,
      value_vars=emission_year_cols,
      var_name='Year',
      value_name='Emission_Coverage_%'
  )

In [138]:
emissions_long['Year'] = emissions_long['Year'].str.replace('.0', '').astype(int)

In [139]:
print(f"To: {len(emissions_long)} rows (long format)")
print(f"Expected: {len(master_df)} × {len(emission_year_cols)} = {len(master_df) * len(emission_year_cols)}")

To: 2808 rows (long format)
Expected: 78 × 36 = 2808


In [140]:
zeros_count = (emissions_long['Emission_Coverage_%'] == 0).sum()
nans_count = emissions_long['Emission_Coverage_%'].isna().sum()
positive_count = (emissions_long['Emission_Coverage_%'] > 0).sum()

In [141]:
print(f"\nData integrity - Emission Coverage:")
print(f"  Zeros (inactive policy years): {zeros_count}")
print(f"  NaN (missing data): {nans_count}")
print(f"  Positive values (active): {positive_count}")
print(f"  Total: {zeros_count + nans_count + positive_count} (should equal {len(emissions_long)})")

if zeros_count + nans_count + positive_count == len(emissions_long):
    print("  All values accounted for")
else:
    print("  Value count mismatch!")


Data integrity - Emission Coverage:
  Zeros (inactive policy years): 1944
  NaN (missing data): 0
  Positive values (active): 864
  Total: 2808 (should equal 2808)
  All values accounted for


Unpivoting Carbon Price

In [142]:
print(f"Using {len(price_year_cols)} price year columns")

Using 36 price year columns


In [143]:
price_long = pd.melt(
      master_df,
      id_vars=metadata_cols,
      value_vars=price_year_cols,
      var_name='Year_Price',
      value_name='Carbon_Price_USD'
  )

In [144]:
price_long['Year'] = price_long['Year_Price'].str.extract('(\d{4})').astype(int)
price_long = price_long.drop('Year_Price', axis=1)

In [145]:
print(f"Result: {len(price_long)} rows")
print(f"Expected: {len(master_df)} × {len(price_year_cols)} = {len(master_df) * len(price_year_cols)}")

Result: 2808 rows
Expected: 78 × 36 = 2808


In [146]:
zeros_count = (price_long['Carbon_Price_USD'] == 0).sum()
nans_count = price_long['Carbon_Price_USD'].isna().sum()
positive_count = (price_long['Carbon_Price_USD'] > 0).sum()
print(f"\nData integrity - Carbon Price:")
print(f"  Zeros (no price set): {zeros_count}")
print(f"  NaN (missing data): {nans_count}")
print(f"  Positive values (price set): {positive_count}")
print(f"  Total: {zeros_count + nans_count + positive_count} (should equal {len(price_long)})")
if zeros_count + nans_count + positive_count == len(price_long):
    print("  All values accounted for")
else:
    print("  Value count mismatch!")


Data integrity - Carbon Price:
  Zeros (no price set): 54
  NaN (missing data): 1988
  Positive values (price set): 766
  Total: 2808 (should equal 2808)
  All values accounted for


Unpivoting Revenue

In [147]:
print(f"Using {len(revenue_year_cols)} revenue year columns")

Using 35 revenue year columns


In [148]:
revenue_long = pd.melt(
      master_df,
      id_vars=metadata_cols,
      value_vars=revenue_year_cols,
      var_name='Year_Revenue',
      value_name='Revenue_Million_USD'
  )

In [149]:
last_revenue_year = int(revenue_year_cols[-1].split('.')[0])

In [150]:
revenue_long['Year'] = revenue_long['Year_Revenue'].str.extract('(\d{4})').astype(int)
revenue_long = revenue_long.drop('Year_Revenue', axis=1)

In [151]:
if last_revenue_year < 2025:
      missing_years = list(range(last_revenue_year + 1, 2026))
      print(f"Adding missing year(s) with NaN: {missing_years}")

      for year in missing_years:
          year_rows = emissions_long[emissions_long['Year'] == year][metadata_cols + ['Year']].copy()
          year_rows['Revenue_Million_USD'] = np.nan
          revenue_long = pd.concat([revenue_long, year_rows], ignore_index=True)
          print(f"  Added {len(year_rows)} rows for year {year}")

print(f"Result after adding missing years: {len(revenue_long)} rows")

Adding missing year(s) with NaN: [2025]
  Added 78 rows for year 2025
Result after adding missing years: 2808 rows


Merging emissions, price, and revenue into one dataset...

In [152]:
master_long = emissions_long.merge(
      price_long,
      on=metadata_cols + ['Year'],
      how='left'
  )
print(f"After merging price: {master_long.shape}")

After merging price: (2808, 10)


In [153]:
master_long = master_long.merge(
      revenue_long,
      on=metadata_cols + ['Year'],
      how='left'
  )
print(f"After merging revenue: {master_long.shape}")

After merging revenue: (2808, 11)


In [154]:
expected_rows = len(master_df) * 36  # 78 policies × 36 years (1990-2025)
print(f"\nExpected rows: {expected_rows} ({len(master_df)} policies × 36 years)")
print(f"Actual rows: {len(master_long)}")

if len(master_long) == expected_rows:
    print("PERFECT! Exact match - no data loss")
else:
    diff = len(master_long) - expected_rows
    print(f"Row count difference: {diff}")


Expected rows: 2808 (78 policies × 36 years)
Actual rows: 2808
PERFECT! Exact match - no data loss


In [155]:
duplicates = master_long.duplicated(subset=['Jurisdiction', 'Year', 'Instrument name']).sum()
print(f"\nDuplicate rows: {duplicates} (should be 0)")
if duplicates == 0: 
    print("No duplicates found")
else:
    print("Duplicates exist!")


Duplicate rows: 0 (should be 0)
No duplicates found


In [156]:
print(f"\nZeros preserved (inactive policy-years):")
print(f"  Emission Coverage: {(master_long['Emission_Coverage_%'] == 0).sum()}")
print(f"  Carbon Price: {(master_long['Carbon_Price_USD'] == 0).sum()}")
print(f"  Revenue: {(master_long['Revenue_Million_USD'] == 0).sum()}")


Zeros preserved (inactive policy-years):
  Emission Coverage: 1944
  Carbon Price: 54
  Revenue: 352


In [157]:
print("\n1. Sweden 2023 (should have data):")
sample1 = master_long[(master_long['Jurisdiction'] == 'Sweden') & (master_long['Year'] == 2023)]
if len(sample1) > 0:
    row = sample1.iloc[0]
    print(f"   Policy: {row['Instrument name']}")
    print(f"   Type: {row['Type']}")
    print(f"   Coverage: {row['Emission_Coverage_%']}%")
    print(f"   Price: ${row['Carbon_Price_USD']}")
    print(f"   Revenue: ${row['Revenue_Million_USD']}M")
else:
    print("   NOT FOUND!")


1. Sweden 2023 (should have data):
   Policy: Sweden carbon tax
   Type: Carbon tax
   Coverage: 0.0003709631193737%
   Price: $125.5565536
   Revenue: $2073.560718M


In [158]:
sample2 = master_long[(master_long['Jurisdiction'] == 'Sweden') & (master_long['Year'] == 1990)]
if len(sample2) > 0:
    row = sample2.iloc[0]
    print(f"   Coverage: {row['Emission_Coverage_%']}")
    print(f"   Price: ${row['Carbon_Price_USD']}")
    print(f"   Revenue: ${row['Revenue_Million_USD']}M")
    if row['Emission_Coverage_%'] == 0:
        print("   Zero preserved (inactive year)")
else:
    print("  NOT FOUND!")

   Coverage: 0.0
   Price: $nan
   Revenue: $nanM
   Zero preserved (inactive year)


In [159]:
print("\n3. Sweden 2025 (Revenue should be NaN):")
sample3 = master_long[(master_long['Jurisdiction'] == 'Sweden') & (master_long['Year'] == 2025)]
if len(sample3) > 0:
    row = sample3.iloc[0]
    print(f"   Coverage: {row['Emission_Coverage_%']}")
    print(f"   Price: ${row['Carbon_Price_USD']}")
    print(f"   Revenue: {row['Revenue_Million_USD']}")
    if pd.isna(row['Revenue_Million_USD']):
          print("   Revenue is NaN (as expected - no 2025 data)")
else:
    print("    NOT FOUND!")


3. Sweden 2025 (Revenue should be NaN):
   Coverage: 0.0003709631193737
   Price: $144.6246989
   Revenue: nan
   Revenue is NaN (as expected - no 2025 data)


In [160]:
master_long['lookup_entity'] = master_long['Jurisdiction'].apply(
      lambda x: get_lookup_entity(x, 'energy')
  )

In [161]:
master_long = master_long.merge(
      energy_mix_clean[['Entity', 'Year',
                        'Coal per capita (kWh)',
                        'Oil per capita (kWh)',
                        'Gas per capita (kWh)',
                        'Nuclear per capita (kWh - equivalent)',
                        'Hydro per capita (kWh - equivalent)',
                        'Wind per capita (kWh - equivalent)',
                        'Solar per capita (kWh - equivalent)',
                        'Other renewables per capita (kWh - equivalent)']],
      left_on=['lookup_entity', 'Year'],
      right_on=['Entity', 'Year'],
      how='left'
  )

In [162]:
master_long = master_long.drop(['lookup_entity', 'Entity'], axis=1)

In [163]:
print(f"\After Energy Mix merge: {master_long.shape}")

\After Energy Mix merge: (2808, 19)


In [164]:
print("Calculating Total Energy per capita...")
master_long['Total_Energy_per_capita'] = (
      master_long['Coal per capita (kWh)'].fillna(0) +
      master_long['Oil per capita (kWh)'].fillna(0) +
      master_long['Gas per capita (kWh)'].fillna(0) +
      master_long['Nuclear per capita (kWh - equivalent)'].fillna(0) +
      master_long['Hydro per capita (kWh - equivalent)'].fillna(0) +
      master_long['Wind per capita (kWh - equivalent)'].fillna(0) +
      master_long['Solar per capita (kWh - equivalent)'].fillna(0) +
      master_long['Other renewables per capita (kWh - equivalent)'].fillna(0)
)

Calculating Total Energy per capita...


In [165]:
print("Calculating Fossil Fuel Dependency %...")
master_long['Fossil_Fuel_Dependency_%'] = np.where(
      master_long['Total_Energy_per_capita'] > 0,
      ((master_long['Coal per capita (kWh)'].fillna(0) +
        master_long['Oil per capita (kWh)'].fillna(0) +
        master_long['Gas per capita (kWh)'].fillna(0)) /
       master_long['Total_Energy_per_capita'] * 100),
      np.nan
  )

Calculating Fossil Fuel Dependency %...


In [166]:
master_long.shape

(2808, 21)

In [167]:
nan_count = master_long['Fossil_Fuel_Dependency_%'].isna().sum()
nan_pct = (nan_count / len(master_long)) * 100
print(f"Fossil_Fuel_Dependency_% NaN count: {nan_count} / {len(master_long)} ({nan_pct:.1f}%)")

Fossil_Fuel_Dependency_% NaN count: 78 / 2808 (2.8%)


In [168]:
for year_range, label in [((1990, 1999), '1990s'), ((2000, 2009), '2000s'),
                             ((2010, 2019), '2010s'), ((2020, 2023), '2020-2023'),
                             ((2024, 2025), '2024-2025')]:
      mask = (master_long['Year'] >= year_range[0]) & (master_long['Year'] <= year_range[1])
      range_data = master_long[mask]
      range_nan = range_data['Fossil_Fuel_Dependency_%'].isna().sum()
      range_pct = (range_nan / len(range_data)) * 100 if len(range_data) > 0 else 0
      print(f"  {label}: {range_nan}/{len(range_data)} NaN ({range_pct:.1f}%)")

  1990s: 0/780 NaN (0.0%)
  2000s: 0/780 NaN (0.0%)
  2010s: 0/780 NaN (0.0%)
  2020-2023: 0/312 NaN (0.0%)
  2024-2025: 78/156 NaN (50.0%)


In [169]:
test_cases = [
      ('Sweden', 2023, 'Direct match'),
      ('Alberta', 2023, 'Parent country (Canada)'),
      ('Albania', 2023, 'Regional fallback (Europe)')
  ]

for jurisdiction, year, description in test_cases:
    row = master_long[(master_long['Jurisdiction'] == jurisdiction) &
                        (master_long['Year'] == year)]
    if len(row) > 0:
        row = row.iloc[0]
        print(f"\n{jurisdiction} {year} ({description}):")
        print(f"  Fossil Fuel %: {row['Fossil_Fuel_Dependency_%']:.1f}%" if pd.notna(row['Fossil_Fuel_Dependency_%']) else "  Fossil Fuel %: NaN")
        print(f"  Total Energy: {row['Total_Energy_per_capita']:.1f} kWh/capita" if pd.notna(row['Total_Energy_per_capita']) else "  Total Energy: NaN")
        print(f"  Coal: {row['Coal per capita (kWh)']:.1f}" if pd.notna(row['Coal per capita (kWh)']) else "  Coal: NaN")
    else:
        print(f"\n{jurisdiction} {year}: NOT FOUND!")


Sweden 2023 (Direct match):
  Fossil Fuel %: 27.9%
  Total Energy: 53449.0 kWh/capita
  Coal: 1767.2

Alberta 2023 (Parent country (Canada)):
  Fossil Fuel %: 67.1%
  Total Energy: 96281.2 kWh/capita
  Coal: 2620.1

Albania 2023 (Regional fallback (Europe)):
  Fossil Fuel %: 75.1%
  Total Energy: 37729.0 kWh/capita
  Coal: 3852.0


In [170]:
print("\nApplying location mapping for population...")
master_long['lookup_entity'] = master_long['Jurisdiction'].apply(
      lambda x: get_lookup_entity(x, 'population')
  )


Applying location mapping for population...


In [171]:
print("Merging population data on [lookup_entity, Year]...")
master_long = master_long.merge(
      population_clean[['Entity', 'Year', 'Population']],
      left_on=['lookup_entity', 'Year'],
      right_on=['Entity', 'Year'],
      how='left'
  )

Merging population data on [lookup_entity, Year]...


In [172]:
master_long = master_long.drop(['lookup_entity', 'Entity'], axis=1)

In [173]:
master_long.shape

(2808, 22)

In [174]:
print("\nCalculating Population_Log feature...")
master_long['Population_Log'] = np.where(
      master_long['Population'].notna(),
      np.log(master_long['Population'] + 1),
      np.nan
  )


Calculating Population_Log feature...


In [175]:
master_long.shape

(2808, 23)

In [176]:
nan_count = master_long['Population'].isna().sum()
nan_pct = (nan_count / len(master_long)) * 100
print(f"Population NaN count: {nan_count} / {len(master_long)} ({nan_pct:.1f}%)")

Population NaN count: 190 / 2808 (6.8%)


In [177]:
for year_range, label in [((1990, 1999), '1990s'), ((2000, 2009), '2000s'),
                             ((2010, 2019), '2010s'), ((2020, 2023), '2020-2023'),
                             ((2024, 2025), '2024-2025')]:
      mask = (master_long['Year'] >= year_range[0]) & (master_long['Year'] <= year_range[1])
      range_data = master_long[mask]
      range_nan = range_data['Population'].isna().sum()
      range_pct = (range_nan / len(range_data)) * 100 if len(range_data) > 0 else 0
      print(f"  {label}: {range_nan}/{len(range_data)} NaN ({range_pct:.1f}%)")

  1990s: 10/780 NaN (1.3%)
  2000s: 10/780 NaN (1.3%)
  2010s: 10/780 NaN (1.3%)
  2020-2023: 4/312 NaN (1.3%)
  2024-2025: 156/156 NaN (100.0%)


In [178]:
test_cases = [
      ('Sweden', 2023, 'Direct match'),
      ('Alberta', 2023, 'Parent country (Canada)'),
      ('Albania', 2023, 'Direct match')
  ]

for jurisdiction, year, description in test_cases:
    row = master_long[(master_long['Jurisdiction'] == jurisdiction) &
                        (master_long['Year'] == year)]
    if len(row) > 0:
        row = row.iloc[0]
        print(f"\n{jurisdiction} {year} ({description}):")
        print(f"  Population: {row['Population']:,.0f}" if pd.notna(row['Population']) else "  Population: NaN")
        print(f"  Population_Log: {row['Population_Log']:.2f}" if pd.notna(row['Population_Log']) else "  Population_Log: NaN")
    else:
        print(f"\n{jurisdiction} {year}: NOT FOUND!")


Sweden 2023 (Direct match):
  Population: 10,551,493
  Population_Log: 16.17

Alberta 2023 (Parent country (Canada)):
  Population: 39,299,098
  Population_Log: 17.49

Albania 2023 (Direct match):
  Population: 2,811,660
  Population_Log: 14.85


In [179]:
master_long['lookup_entity'] = master_long['Jurisdiction'].apply(
      lambda x: get_lookup_entity(x, 'co2')
  )

In [180]:
master_long = master_long.merge(
      co2_clean[['Entity', 'Year', 'Annual_CO2_emissions']],
      left_on=['lookup_entity', 'Year'],
      right_on=['Entity', 'Year'],
      how='left'
  )

In [181]:
master_long = master_long.drop(['lookup_entity', 'Entity'], axis=1)

In [182]:
nan_count = master_long['Annual_CO2_emissions'].isna().sum()
nan_pct = (nan_count / len(master_long)) * 100
print(f"Annual_CO2_emissions NaN count: {nan_count} / {len(master_long)} ({nan_pct:.1f}%)")

Annual_CO2_emissions NaN count: 78 / 2808 (2.8%)


In [183]:
for year_range, label in [((1990, 1999), '1990s'), ((2000, 2009), '2000s'),
                             ((2010, 2019), '2010s'), ((2020, 2023), '2020-2023'),
                             ((2024, 2025), '2024-2025')]:
      mask = (master_long['Year'] >= year_range[0]) & (master_long['Year'] <= year_range[1])
      range_data = master_long[mask]
      range_nan = range_data['Annual_CO2_emissions'].isna().sum()
      range_pct = (range_nan / len(range_data)) * 100 if len(range_data) > 0 else 0
      print(f"  {label}: {range_nan}/{len(range_data)} NaN ({range_pct:.1f}%)")

  1990s: 0/780 NaN (0.0%)
  2000s: 0/780 NaN (0.0%)
  2010s: 0/780 NaN (0.0%)
  2020-2023: 0/312 NaN (0.0%)
  2024-2025: 78/156 NaN (50.0%)


In [184]:
test_cases = [
      ('Sweden', 2023, 'Direct match'),
      ('Alberta', 2023, 'Parent country (Canada)'),
      ('Albania', 2023, 'Direct match')
  ]

for jurisdiction, year, description in test_cases:
    row = master_long[(master_long['Jurisdiction'] == jurisdiction) &
                        (master_long['Year'] == year)]
    if len(row) > 0:
        row = row.iloc[0]
        print(f"\n{jurisdiction} {year} ({description}):")
        if pd.notna(row['Annual_CO2_emissions']):
            co2_millions = row['Annual_CO2_emissions'] / 1_000_000
            print(f"  CO2: {row['Annual_CO2_emissions']:,.0f} tonnes ({co2_millions:.1f}M tonnes)")
        else:
            print(f"  CO2: NaN")
    else:
        print(f"\n{jurisdiction} {year}: NOT FOUND!")


Sweden 2023 (Direct match):
  CO2: 36,708,596 tonnes (36.7M tonnes)

Alberta 2023 (Parent country (Canada)):
  CO2: 545,478,500 tonnes (545.5M tonnes)

Albania 2023 (Direct match):
  CO2: 4,417,333 tonnes (4.4M tonnes)


In [185]:
test_lookup = get_lookup_entity('EU', 'gdp')
print(f"\n'EU' now maps to: '{test_lookup}'")
print(f"'{test_lookup}' exists in gdp_clean: {test_lookup in gdp_clean['Entity'].values}")


'EU' now maps to: 'European Union (27)'
'European Union (27)' exists in gdp_clean: True


In [186]:
master_long['lookup_entity'] = master_long['Jurisdiction'].apply(
      lambda x: get_lookup_entity(x, 'gdp')
  )

In [187]:
master_long = master_long.merge(
      gdp_clean[['Entity', 'Year', 'GDP']],
      left_on=['lookup_entity', 'Year'],
      right_on=['Entity', 'Year'],
      how='left'
  )

In [188]:
master_long = master_long.drop(['lookup_entity', 'Entity'], axis=1)

In [189]:
master_long.shape

(2808, 25)

In [190]:
nan_count = master_long['GDP'].isna().sum()
nan_pct = (nan_count / len(master_long)) * 100
print(f"\nGDP NaN count: {nan_count} / {len(master_long)} ({nan_pct:.1f}%)")


GDP NaN count: 190 / 2808 (6.8%)


In [191]:
for year_range, label in [((1990, 1999), '1990s'), ((2000, 2009), '2000s'),
                             ((2010, 2019), '2010s'), ((2020, 2023), '2020-2023'),
                             ((2024, 2025), '2024-2025')]:
      mask = (master_long['Year'] >= year_range[0]) & (master_long['Year'] <= year_range[1])
      range_data = master_long[mask]
      range_nan = range_data['GDP'].isna().sum()
      range_pct = (range_nan / len(range_data)) * 100 if len(range_data) > 0 else 0
      print(f"  {label}: {range_nan}/{len(range_data)} NaN ({range_pct:.1f}%)")

  1990s: 10/780 NaN (1.3%)
  2000s: 10/780 NaN (1.3%)
  2010s: 10/780 NaN (1.3%)
  2020-2023: 4/312 NaN (1.3%)
  2024-2025: 156/156 NaN (100.0%)


In [192]:
print("\nJurisdictions with missing GDP (any year):")
missing_gdp_jurisdictions = master_long[master_long['GDP'].isna()]['Jurisdiction'].unique()
print(f"  Count: {len(missing_gdp_jurisdictions)}")
if len(missing_gdp_jurisdictions) <= 10:
      for j in missing_gdp_jurisdictions:
          print(f"    - {j}")


Jurisdictions with missing GDP (any year):
  Count: 68


In [193]:
test_cases = [
      ('Sweden', 2023, 'Direct match'),
      ('Alberta', 2023, 'Parent country (Canada)'),
      ('Albania', 2023, 'Direct match'),
      ('EU', 2023, 'Special case (EU calculated)')
  ]

for jurisdiction, year, description in test_cases:
    row = master_long[(master_long['Jurisdiction'] == jurisdiction) &
                        (master_long['Year'] == year)]
    if len(row) > 0:
        row = row.iloc[0]
        print(f"\n{jurisdiction} {year} ({description}):")
        if pd.notna(row['GDP']):
            gdp_billions = row['GDP'] / 1_000_000_000
            gdp_trillions = row['GDP'] / 1_000_000_000_000
            if gdp_trillions >= 1:
                print(f"  GDP: ${row['GDP']:,.0f} (${gdp_trillions:.2f}T)")
            else:
                print(f"  GDP: ${row['GDP']:,.0f} (${gdp_billions:.1f}B)")
        else:
            print(f"  GDP: NaN")
    else:
        print(f"\n{jurisdiction} {year}: NOT FOUND!")


Sweden 2023 (Direct match):
  GDP: $628,732,125,000 ($628.7B)

Alberta 2023 (Parent country (Canada)):
  GDP: $2,165,437,000,000 ($2.17T)

Albania 2023 (Direct match):
  GDP: $44,899,289,062 ($44.9B)

EU 2023 (Special case (EU calculated)):
  GDP: $23,014,355,519,529 ($23.01T)


In [197]:
print(f"\nFinal master_long shape: {master_long.shape}")


Final master_long shape: (2808, 25)


In [198]:
print(f"\nAll {len(master_long.columns)} columns:")
for i, col in enumerate(master_long.columns, 1):
    print(f"  {i:2d}. {col}")


All 25 columns:
   1. Unique ID
   2. Instrument name
   3. Type
   4. Status
   5. Jurisdiction
   6. Region
   7. Income group
   8. Year
   9. Emission_Coverage_%
  10. Carbon_Price_USD
  11. Revenue_Million_USD
  12. Coal per capita (kWh)
  13. Oil per capita (kWh)
  14. Gas per capita (kWh)
  15. Nuclear per capita (kWh - equivalent)
  16. Hydro per capita (kWh - equivalent)
  17. Wind per capita (kWh - equivalent)
  18. Solar per capita (kWh - equivalent)
  19. Other renewables per capita (kWh - equivalent)
  20. Total_Energy_per_capita
  21. Fossil_Fuel_Dependency_%
  22. Population
  23. Population_Log
  24. Annual_CO2_emissions
  25. GDP


In [205]:
print(f"Duplicates: {master_long.duplicated(subset=['Jurisdiction', 'Year']).sum()}")

Duplicates: 360


In [204]:
master_long.head(100)

,Unique ID,Instrument name,Type,Status,Jurisdiction,Region,Income group,Year,Emission_Coverage_%,Carbon_Price_USD,...,Hydro per capita (kWh - equivalent),Wind per capita (kWh - equivalent),Solar per capita (kWh - equivalent),Other renewables per capita (kWh - equivalent),Total_Energy_per_capita,Fossil_Fuel_Dependency_%,Population,Population_Log,Annual_CO2_emissions,GDP
0,Tax_AL,Albania carbon tax,Carbon tax,Implemented,Albania,Europe & Central Asia,Upper middle income,1990,0.000000,NaN,...,2475.444800,2.986670,0.047724,81.682000,48572.274394,86.738952,3277965.0,15.002734,5.520602e+06,1.281962e+10
1,Tax_CA_Alberta,Alberta carbon tax,Carbon tax,Abolished,Alberta,North America,High income,1990,0.000000,NaN,...,29555.926000,0.191839,0.000000,444.638370,106191.335709,64.927726,27789437.0,17.140167,4.580179e+08,1.051178e+12
2,ETS_CA_Alberta,Alberta TIER,ETS,Implemented,Alberta,North America,High income,1990,0.000000,NaN,...,29555.926000,0.191839,0.000000,444.638370,106191.335709,64.927726,27789437.0,17.140167,4.580179e+08,1.051178e+12
3,Tax_AR,Argentina carbon tax,Carbon tax,Implemented,Argentina,Latin America & Caribbean,Upper middle income,1990,0.000000,NaN,...,1502.118800,0.000000,0.000000,12.690632,15569.930292,86.305299,32755907.0,17.304594,1.121883e+08,2.279555e+11
4,ETS_AU1,Australia CPM,ETS,Abolished,Australia,East Asia & Pacific,High income,1990,0.000000,NaN,...,2393.897000,0.000000,0.000000,138.584350,61214.798350,95.862959,17126304.0,16.656126,2.780610e+08,5.687269e+11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Tax_CO,Colombia carbon tax,Carbon tax,Implemented,Colombia,Latin America & Caribbean,Upper middle income,1991,0.000000,NaN,...,2326.908400,0.000000,0.000000,25.397774,7679.590974,69.369382,33098373.0,17.314995,5.660903e+07,2.822566e+11
96,Tax_DK,Denmark carbon tax,Carbon tax,Implemented,Denmark,Europe & Central Asia,High income,1991,0.000000,NaN,...,13.688556,398.854340,0.000000,215.230990,45144.943886,98.609426,5154346.0,15.455351,6.410461e+07,1.676362e+11
97,Tax_EE,Estonia carbon tax,Carbon tax,Implemented,Estonia,Europe & Central Asia,High income,1991,0.000000,NaN,...,0.000000,0.000000,0.000000,0.000000,74582.455000,100.000000,1565380.0,14.263640,3.413543e+07,2.316718e+10
98,ETS_EU,EU ETS,ETS,Implemented,EU,Europe & Central Asia,High income,1991,0.000000,NaN,...,1963.676300,6.396560,0.089117,132.586290,41505.460267,83.031579,NaN,NaN,3.804530e+09,1.047885e+13


In [208]:
print(f"\n1. Shape: {master_long.shape}")
print(f"   Expected: (2808, 25)")
print(f"   Status: {'PASS' if master_long.shape == (2808, 25) else 'FAIL'}")


1. Shape: (2808, 25)
   Expected: (2808, 25)
   Status: PASS


In [209]:
dups = master_long.duplicated(subset=['Jurisdiction', 'Year', 'Instrument name']).sum()
print(f"\n2. Duplicates: {dups}")
print(f"   Expected: 0")
print(f"   Status: {'PASS' if dups == 0 else 'FAIL'}")


2. Duplicates: 0
   Expected: 0
   Status: PASS


In [210]:
training = master_long[(master_long['Year'] >= 1990) & (master_long['Year'] <= 2023)]
fossil_pct = (training['Fossil_Fuel_Dependency_%'].notna().sum() / len(training)) * 100
pop_pct = (training['Population_Log'].notna().sum() / len(training)) * 100

In [211]:
print(f"\n3. ML Features (1990-2023):")
print(f"   Fossil_Fuel_%: {fossil_pct:.1f}% complete")
print(f"   Population_Log: {pop_pct:.1f}% complete")
print(f"   Status: {'PASS' if fossil_pct > 85 and pop_pct > 85 else 'FAIL'}")


3. ML Features (1990-2023):
   Fossil_Fuel_%: 100.0% complete
   Population_Log: 98.7% complete
   Status: PASS


In [212]:
eu_gdp = master_long[(master_long['Jurisdiction'] == 'EU') & (master_long['Year'] == 2023)]['GDP'].iloc[0]      
print(f"\n4. EU GDP (2023): ${eu_gdp/1e12:.2f}T")
print(f"   Status: {'PASS' if pd.notna(eu_gdp) else 'FAIL'}")


4. EU GDP (2023): $23.01T
   Status: PASS


In [213]:
if master_long.shape == (2808, 25) and dups == 0 and fossil_pct > 85 and pop_pct > 85 and pd.notna(eu_gdp):     
    print("ALL VALIDATIONS PASSED - DATASET READY FOR MODEL TRAINING")
else:
    print("SOME VALIDATIONS FAILED - NEEDS REVIEW")

ALL VALIDATIONS PASSED - DATASET READY FOR MODEL TRAINING


### Merged Dataset

Saving Merged Dataset

In [214]:
master_long_full = master_long.copy()

In [217]:
output_full = '../dataset/processed/ecoimpact_complete_dataset.csv'
master_long_full.to_csv(output_full, index=False)
print(f"Full dataset saved to: {output_full}")

Full dataset saved to: ../dataset/processed/ecoimpact_complete_dataset.csv


In [218]:
print(f"   Rows: {len(master_long_full)} (78 policies × 36 years)")

   Rows: 2808 (78 policies × 36 years)


In [219]:
print(f"   Columns: {master_long_full.shape[1]}")

   Columns: 25


#### Summary: External Dataset Integration

  We merged 4 external datasets with our master dataset to add country-specific features for better ML predictions.

  ##### What we did:

  **1. Prepared external datasets**
  - Renamed columns to consistent names (Population, Annual_CO2_emissions, GDP)
  - Calculated EU27 GDP (sum of 27 member states)
  - Verified year ranges: Energy Mix (1965-2024), Population/GDP (1950-2023), CO2 (1750-2024)

  **2. Created location mapping function**
  - **Direct match**: Sweden → Sweden (34-38 jurisdictions)
  - **Parent country**: Alberta → Canada (30 sub-national regions)
  - **Special cases**: EU → European Union (27), Korea Rep. → South Korea
  - **Regional fallback**: Albania → Europe (4 countries, energy data only)
  - Result: 100% coverage for all 68 jurisdictions

  **3. Merged datasets one by one**
  - Energy Mix: Added 8 energy sources, calculated `Fossil_Fuel_Dependency_%` = (Coal + Oil + Gas) / Total × 100  
  - Population: Added population, calculated `Population_Log` = log(population + 1)
  - CO2 Emissions: Added `Annual_CO2_emissions` for calculating actual tonnes reduced
  - GDP: Added GDP for contextual display (revenue as % of GDP)

  **4. Result**
  - Started with: 2,808 rows × 11 columns (master dataset unpivoted)
  - Ended with: 2,808 rows × 25 columns (master + external features)
  - No duplicates, no data loss
  - Training period (1990-2023): 100% complete for Fossil_Fuel_%, 98.7% for Population_Log

  **5. Output file**
  - Saved as: `ecoimpact_complete_dataset.csv`
  - Ready for ML model training

 